In [14]:
:dep itertools
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}
:dep log
:dep env_logger

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use itertools::Itertools;
use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;
use log::{LevelFilter, info, warn};
use env_logger::Env;

use vawt::areofoil::Aerofoil;
use vawt::{VAWTSolver, VAWTSolution};

In [15]:
env_logger::init();
log::set_max_level(LevelFilter::Info);

In [16]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [29]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;
let aerofoil: &'static Aerofoil = Box::leak(Box::new(aerofoil));
let n = 100;
let mut solver = VAWTSolver::new(&aerofoil);
solver.re(31_300.0)
    .solidity(0.3525)
    .n_streamtubes(n)
    .epsilon(0.0001);

In [30]:
let solution = solver.tsr(3.25).solve_optimize_beta();

let d_t_half = PI / n as f64;
let theta = Array::linspace(d_t_half, 2.0 * PI - d_t_half, n);

evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..2.0, -1f64..1f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}π", x))
        .draw()?;

    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.a(theta))), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.c_tan(theta))), 
        &BLUE
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta / PI, solution.beta(theta))),
        &GREEN
    ))?;
    Ok(())
})


-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
0.0π
 
 
 
0.5π
 
 
 
1.0π
 
 
 
1.5π
 
 
 
2.0π
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="42,149 48,137 54,175 60,180 66,180 72,178 78,177 84,176 90,176 96,177 102,178 108,179 114,180 120,180 126,181 132,182 138,183 144,183 150,184 156,183 162,183 168,183 174,182 180,183 186,183 192,183 198,184 204,184 210,184 216,185 222,185 228,186 234,187 240,188 246,188 252,189 258,189 264,191 270,192 276,193 282,193 288,193 294,193 300,194 306,195 312,198 318,200 324,205 330,213 336,232 342,227 348,208 354,193 360,184 366,177 372,171 378,167 384,163 390,161 396,159 402,158 408,157 414,156 420,156 426,155 432,155 438,153 444,153 450,153 456,153 462,152 468,153 474,151 480,152 486,152 492,152 498,151 504,151 510,151 516,151 522,150 528,150 534,148 540,149 546,149 552,149 558,148 564,148 570,148 576,148 582,148 588,149 594,147 600,146 606,146 612,145 618,139 624,108 630,0 636,0 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="42,223 48,223 54,222 60,221 66,220 72,219 78,218 84,216 90,215 96,214 102,213 108,212 114,211 120,210 126,209 132,209 138,208 144,207 150,206 156,205 162,204 168,203 174,202 180,201 186,201 192,200 198,200 204,199 210,198 216,198 222,198 228,198 234,198 240,199 246,199 252,199 258,200 264,201 270,202 276,204 282,205 288,206 294,208 300,210 306,212 312,215 318,217 324,220 330,222 336,224 342,224 348,222 354,221 360,219 366,218 372,217 378,217 384,216 390,215 396,214 402,213 408,213 414,213 420,213 426,213 432,213 438,213 444,213 450,213 456,213 462,213 468,214 474,214 480,214 486,214 492,215 498,215 504,215 510,215 516,216 522,216 528,216 534,217 540,217 546,218 552,218 558,219 564,219 570,220 576,220 582,220 588,221 594,221 600,222 606,222 612,222 618,222 624,223 630,224 636,223 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="42,219 48,205 54,214 60,216 66,218 72,219 78,221 84,223 90,225 96,228 102,230 108,233 114,235 120,238 126,240 132,242 138,245 144,247 150,249 156,250 162,251 168,252 174,253 180,254 186,254 192,255 198,256 204,256 210,256 216,256 222,256 228,255 234,255 240,254 246,253 252,251 258,249 264,248 270,246 276,243 282,240 288,236 294,232 300,228 306,224 312,221 318,217 324,215 330,216 336,219 342,225 348,230 354,231 360,229 366,227 372,225 378,223 384,221 390,220 396,218 402,216 408,213 414,212 420,210 426,209 432,207 438,207 444,206 450,205 456,204 462,204 468,203 474,203 480,203 486,203 492,203 498,203 504,204 510,204 516,205 522,206 528,207 534,208 540,208 546,209 552,210 558,211 564,212 570,213 576,214 582,215 588,216 594,218 600,219 606,220 612,222 618,223 624,226 630,224 636,221 "/>

In [33]:
let tsr = Array::linspace(1.0,5.0,30);
let cps = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta(0.0).ct_cp().1
    })
);
let cps_opt = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta_fn(|theta| solution.beta(theta)).ct_cp().1
    })
);

In [34]:
evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 0f64..0.6)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .draw()?;

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps.iter().cloned()), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps_opt.iter().cloned()),
        &BLUE
    ))?;
    Ok(())
})

0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0